Lets write the barbershop backend in here as practice

In [1]:
import pandas as pd
import numpy as np

try:
    from ipywidgets import interact
    import ipywidgets as widgets
    from bokeh.io import push_notebook, show, output_notebook
    from bokeh.plotting import figure, ColumnDataSource
    from bokeh.models import Span, LogColorMapper, ColumnDataSource, ColorBar
    from bokeh.layouts import row
    from bokeh.palettes import Viridis256, Spectral6
    from bokeh.models.tools import HoverTool
    from IPython.display import display
    from bokeh.transform import linear_cmap
    output_notebook()   

except ImportError:
    raise ImportError('Please install Bokeh and ipywidgets')

Loading BokehJS ...

In [2]:
from astropy.table import Table
def read_kepler(size=1000):
#     data = Table.read('/home/andy/Documents/Research/HackDay/kepler_dr2_1arcsec.fits', format='fits')
    data = Table.read('/Users/Oliver/PhD/Gaia_Project/data/KepxDR2/kepler_dr2_1arcsec.fits')
    kdf = data.to_pandas()
    kdf.rename(columns={'kepid':'KICID'},inplace=True)
    kdf = kdf[kdf.parallax > 0.]
    
    if size == 'all':
        return kdf
    elif size == 'smaller':
        return kdf[(kdf.ra < 284.5) & (kdf.dec > 41.5) & (kdf.dec < 45)]
    else:
        return kdf[:size]
df = read_kepler(size='smaller')
print(len(df))

8148


In [18]:
def update(low_gmag, colour):
    outdf = df[df.phot_g_mean_mag > low_gmag]
    r.data_source.data = dict(x=outdf['ra'],
                                y = outdf['dec'],
                                z = outdf[colour],
                                radius = outdf['radius'])
    
    mapper = linear_cmap(field_name='z', palette=Viridis256 ,low=min(outdf[colour]) ,high=max(outdf[colour]))
    color_bar = ColorBar(color_mapper=mapper['transform'], width=10,  location=(0,0), title='Gmag')
    p.add_layout(color_bar, 'right')
    push_notebook()  

In [20]:
#Create the data array
x = df.ra
y = df.dec
z = df.phot_g_mean_mag
df['radius'] = np.log10(df.parallax)/20
radius = df.radius
source = ColumnDataSource(dict(x=x,y=y,z=z,radius=radius))

# #Create the colourmap
# mapper = linear_cmap(field_name='z', palette=Viridis256 ,low=min(z) ,high=max(z))

#Draw the figure and plot
p = figure(plot_width=600, plot_height=600, 
           tools='pan,box_zoom,reset,save,box_select,lasso_select',
           title="test")
r = p.circle(x='x', y='y', line_color=mapper,color=mapper, fill_alpha=1, radius='radius', source=source)
# color_bar = ColorBar(color_mapper=mapper['transform'], width=10,  location=(0,0), title='Gmag')
# p.add_layout(color_bar, 'right')

#Draw the updater
ui = interact(update, colour=['phot_g_mean_mag'],low_gmag=(z.min(), z.max()))
show(p, notebook_handle=True)
display(ui)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTEyLjMxNzEzMTA0MjQ4MDQ2OSwgZGVzY3JpcHRpb249dSdsb3dfZ21hZycsIG1heD0xOS45NzIxMjYwMDcwODAwNzgsIG3igKY=


<function __main__.update>

In [6]:
df.parallax.max()

52.13743702398894